In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import cv2
import requests
from collections import Counter
import pickle
import pandas as pd
import sqlite3
import re
import csv
import time
import numpy as np

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
options = uc.ChromeOptions()
options.headless = False   # bật trình duyệt để dễ debug
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/115.0.0.0 Safari/537.36"
)
driver = uc.Chrome(options=options, use_subprocess=False)
url = "https://www.arbeitsagentur.de/jobsuche/suche?angebotsart=4&ausbildungsart=0&arbeitszeit=vz&branche=22;1;2;9;3;5;7;10;11;16;12;21;26;15;17;19;20;8;23;29&veroeffentlichtseit=7&sort=veroeffdatum"
driver.get(url)
driver.implicitly_wait(10)


In [ ]:
#Handle accept cookies pop-up
def accept_cookies_shadow(driver, timeout=10):
    try:
        host = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "bahf-cookie-disclaimer-dpl3"))
        )
        root = driver.execute_script("return arguments[0].shadowRoot", host)
        btn = root.find_element(By.CSS_SELECTOR, "button.ba-btn.ba-btn-contrast.sc-bahf-cd-modal")
        driver.execute_script("arguments[0].click();", btn)
        print("Cookie accepted")
    except Exception as e:
        print("ℹCookie banner not found / already accepted:", e)

time.sleep(2)
accept_cookies_shadow(driver)

✅ Cookie accepted


In [ ]:
#Click button to render new data
def scroll_to_bottom_via_end(driver, idle_rounds=2, pause=1.2, timeout=5):
    wait = WebDriverWait(driver, timeout)
    no_button_count = 0
    while True:
        try:
            button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.ba-btn.ba-btn-tertiary"))
            )
            driver.execute_script("arguments[0].click();", button)
            time.sleep(pause)
            no_button_count = 0  # reset khi click thành công
        except (TimeoutException, NoSuchElementException):
            no_button_count += 1
            if no_button_count >= idle_rounds:
                print("Scroll DONE!!!")
                break

#Parser gape to html to get data faster
def get_listing_links(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    total_links = []
    for a in soup.select("a.ergebnisliste-item"):
        href = a.get("href")
        if href and href not in total_links: 
            total_links.append(href)
    print(f"Total links collected: {len(total_links)}")
    return total_links

#Execute
scroll_to_bottom_via_end(driver, idle_rounds=2, pause=1.2, timeout=5)
job_links = get_listing_links(driver)
print(job_links)

✅ Scroll DONE!!!
Total links collected: 88
['https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203484803-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203481840-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203480967-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203480858-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203480751-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203477191-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203476810-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203476520-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203476511-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203476508-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203476057-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10000-1203474473-S', 'https://www.arbeitsagentur.de/jobsuche/jobdetail/10001-1001659131-S', 'https://www.arbeitsagentur.de/jo

In [ ]:
#Click button reload captcha
def _reload_captcha_and_wait(driver, timeout=10):
    img = driver.find_element(By.ID, "kontaktdaten-captcha-image")
    old_src = img.get_attribute("src")
    reload_btn = driver.find_element(By.ID, "kontaktdaten-captcha-reload-button")
    reload_btn.click()
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: d.find_element(By.ID, "kontaktdaten-captcha-image").get_attribute("src") != old_src
        )
    except TimeoutException:
        time.sleep(1)  

#Click button continue retry input captcha
def _click_retry_if_exists(driver, timeout=3):
    try:
        btn = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((By.ID, "kontaktdaten-fehler-erneut-versuchen-button"))
        )
        btn.click()
        time.sleep(2)  
        return True
    except TimeoutException:
        return False

#Handle captcha img
def solve_captcha(driver, max_attempts=5):
    for attempt in range(max_attempts):
        try:
            #Get img form link
            captcha_img = driver.find_element(By.ID, "kontaktdaten-captcha-image")
            img_url = captcha_img.get_attribute("src")

            response = requests.get(img_url, stream=True, headers={"Cache-Control": "no-cache"})
            if response.status_code != 200:
                raise Exception("Can't get captcha img")

            img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
            img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

            # Pre-processing img
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 5))
            opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

            mask = opening.copy()
            lines = cv2.HoughLinesP(opening, 1, np.pi/180, threshold=50, minLineLength=30, maxLineGap=1)
            if lines is not None:
                for line in lines:
                    x1, y1, x2, y2 = line[0]
                    cv2.line(mask, (x1, y1), (x2, y2), 0, 2)

            # OCR on many img
            images = [img, gray, thresh, opening, mask]
            results = []
            for im in images:
                text = pytesseract.image_to_string(
                    im,
                    config="--psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
                )
                filtered = re.sub(r'[^A-Z0-9]', '', text.strip().upper())
                if len(filtered) >= 6:
                    results.append(filtered[:6])

            if results:
                most_common = Counter(results).most_common(1)[0][0]
                if len(most_common) == 6:
                    return most_common

            #If not have retry button, click reload captcha button
            if not _click_retry_if_exists(driver):
                _reload_captcha_and_wait(driver, timeout=10)

        except Exception as e:
            print(f"Error solve_captcha (attempt {attempt+1}):", e)
            if not _click_retry_if_exists(driver):
                try:
                    _reload_captcha_and_wait(driver, timeout=5)
                except Exception:
                    time.sleep(1.5)
    return None

#Get result of process, if it have 6 letter & repeat many time, try it until captcha disappear
def submit_captcha_until_success(driver, max_submit_attempts=20, wait_timeout=10):
    wait = WebDriverWait(driver, wait_timeout)
    for attempt in range(1, max_submit_attempts + 1):
        code = solve_captcha(driver, max_attempts=5)
        if not code:
            # OCR result not meet requiment, reload
            if not _click_retry_if_exists(driver):
                try:
                    _reload_captcha_and_wait(driver, timeout=10)
                except Exception:
                    time.sleep(1.5)
            continue
        try:
            # Input captcha
            input_box = wait.until(EC.presence_of_element_located((By.ID, "kontaktdaten-captcha-input")))
            input_box.clear()
            input_box.send_keys(code)
            # Click submit
            submit_btn = wait.until(EC.element_to_be_clickable((By.ID, "kontaktdaten-captcha-absenden-button")))
            submit_btn.click()
        except Exception as e:
            print(f"Lỗi khi điền/gửi captcha (attempt {attempt}):", e)
            if not _click_retry_if_exists(driver):
                try:
                    _reload_captcha_and_wait(driver, timeout=10)
                except Exception:
                    time.sleep(1.5)
            continue
        # Check if wrong answer 
        error_locator = (By.ID, "kontaktdaten-captcha-input-fehler")
        try:
            WebDriverWait(driver, 4).until(EC.presence_of_element_located(error_locator))
            # If it wrongs, retry
            if not _click_retry_if_exists(driver):
                _reload_captcha_and_wait(driver, timeout=10)
            continue
        except TimeoutException:
            return code
    return None

In [44]:
def scrape_data(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    # Profession_1
    profession_1 = soup.find("div", id="detail-kopfbereich-titel")
    profession_1 = profession_1.get_text(strip=True) if profession_1 else "N/A"
    # Profession_2
    profession_2 = soup.find("span", id="detail-kopfbereich-ausbildungsberuf")
    profession_2 = profession_2.get_text(strip=True) if profession_2 else "N/A"
    # Salary (list các năm)
    salary_items = soup.select("li[id^=detail-kopfbereich-ausbildungsverguetung-]")
    salary = [item.get_text(" ", strip=True).replace("\xa0", " ") for item in salary_items] if salary_items else []
    # Company Name
    company = soup.find("div", id="detail-kopfbereich-firma")
    company = company.get_text(strip=True) if company else "N/A"
    # Address
    address_el = soup.find("a", id=lambda x: x and x.startswith("detail-arbeitsorte-arbeitsort-"))
    address = address_el.get_text(strip=True) if address_el else "N/A"
    # Start Day
    start_day = soup.find("li", id="detail-kopfbereich-eintrittsdatum-mit-datum")
    start_day = start_day.get_text(strip=True) if start_day else "N/A"
    # Telephone
    telephone = soup.find("a", id=lambda x: x and x.startswith("detail-bewerbung-telefon-Telefon"))
    telephone = telephone.get_text(" ", strip=True).replace("\xa0", " ") if telephone else "N/A"
    # Email
    email = soup.find("a", id=lambda x: x and x.startswith("detail-bewerbung-mail"))
    email = email.get_text(strip=True) if email else "N/A"
    # Job Description
    job_description = soup.find("div", id="detail-beschreibung-beschreibung")
    job_description = job_description.get_text("\n", strip=True) if job_description else "N/A"
    # Reference
    reference = soup.find("span", id="detail-footer-referenznummer")
    reference = reference.get_text(strip=True) if reference else "N/A"
    # External Link
    external_link = soup.find("a", id="detail-beschreibung-externe-url-btn")
    external_link = external_link["href"] if external_link else "N/A"
    # Application Link
    application_link = soup.find("a", id="detail-bewerbung-url")
    application_link = application_link["href"] if application_link else "N/A"
    result = {
        "Profession_1": profession_1,
        "Profession_2": profession_2,
        "Salary": salary,
        "Company": company,
        "Address": address,
        "Start Day": start_day,
        "Telephone": telephone,
        "Email": email,
        "Job Description": job_description,
        "Reference": reference,
        "External Link": external_link,
        "Application Link": application_link
    }
    return result


| Profession | Salary | Company Name | Location | Start Date | Telephone | Email | Job Description | Ref.-Nr. | External Link | Application Link |

In [ ]:
try:
    cookies = pickle.load(open("cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    driver.refresh()
    print("Save cookies.pkl")
except Exception:
    print("No cookies yet, make a new one if pass captcha.")

all_data=[]
# for idx, link in enumerate(job_links):
for idx, link in enumerate(job_links[60:], start=61):
    driver.get(link)
    time.sleep(2.5)
    try:
        captcha_imgs = driver.find_elements(By.ID, "kontaktdaten-captcha-image")
        if captcha_imgs:
            print(f"Captcha at link {idx+1}: {link}")
            result = submit_captcha_until_success(driver, max_submit_attempts=30)
            if result:
                print("Captcha OK:", result)
                with open("cookies.pkl", "wb") as f:
                    pickle.dump(driver.get_cookies(), f)
                print("Save new cookies.pkl")

            else:
                print("Can't pass this link captcha, move on.")
                continue 
    except Exception as e:
        print("Error:", e)
        continue
    try:
        data = scrape_data(driver)
        all_data.append(data)
        print("Data:", data)
    except Exception as e:
        print(f"Scrape failed at {link}:", e)

⚠️ Chưa có cookie lưu sẵn, sẽ tạo mới sau khi vượt captcha.
📄 Data: {'Profession_1': 'Ausbildung Verkäufer (m/w/d) 2025', 'Profession_2': 'Verkäufer/in', 'Salary': ['1. Jahr: 1.250 €/Monat', '2. Jahr: 1.350 €/Monat'], 'Company': 'Arbeitgeber:LIDL Vertriebs-GmbH & Co. KG', 'Address': 'Helenenstr. 66, 45143 Essen, Ruhr', 'Start Day': 'Beginn ab 01.09.2026', 'Telephone': 'N/A', 'Email': 'N/A', 'Job Description': 'Wer mit uns ins Berufsleben startet, ist von Anfang an mittendrin statt nur dabei. Wir wollen, dass du Verantwortung übernimmst und als Teil des Teams nicht nur Neues lernst, sondern auch Neues einbringst. Was dabei natürlich nie zu kurz kommen darf? Der Spaß und deine Zukunftsperspektiven! Und das alles zu einem fairen Gehalt.\nDeine Aufgaben\nIn deiner 2-jährigen Ausbildung lernst du, worauf es im Handel ankommt und wie wir in unseren Lidl-Filialen mit Qualität, Frische und Sauberkeit für ein gelungenes Einkaufserlebnis sorgen\nOb bei der Verräumung und Präsentation der Ware, b

KeyboardInterrupt: 

In [ ]:
# Unnecessary process, just saving data for later processing
if all_data:
    keys = all_data[0].keys()
    with open("jobs_data.csv", "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(all_data)
    print("Save jobs_data.csv")
else:
    print("Nothing to save.")

Đã lưu dữ liệu vào jobs_data.csv


In [ ]:
#Refill email base on "Job Description" and "Application Link"
EMAIL_REGEX = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
df = pd.read_csv("jobs_data.csv", delimiter=";", quotechar='"', encoding="utf-8-sig")
for idx, row in df.iterrows():
    if pd.isna(row["Email"]):  
        email_found = None
        desc = str(row["Job Description"])
        match = re.search(EMAIL_REGEX, desc)
        if match:
            email_found = match.group(0)
        if not email_found and not pd.isna(row["Application Link"]):
            try:
                driver.get(row["Application Link"])
                time.sleep(2)
                page_text = driver.find_element(By.TAG_NAME, "body").text
                match = re.search(EMAIL_REGEX, page_text)
                if match:
                    email_found = match.group(0)
            except Exception as e:
                print(f"Can't scrape data from {row['Application Link']}: {e}")
        if email_found:
            df.at[idx, "Email"] = email_found
            print(f"[{idx}] Found Email: {email_found}")

conn = sqlite3.connect("jobs_data.db") 
df.to_sql("jobs", conn, if_exists="replace", index=False)  
conn.close()

print("Save jobs_data.db")

[1] Tìm thấy Email: nordhorn@rvr.ruhr
[22] Tìm thấy Email: bewerbung@lidl.de
[23] Tìm thấy Email: bewerbung@lidl.de
[24] Tìm thấy Email: bewerbung@lidl.de
[25] Tìm thấy Email: bewerbung@lidl.de
[26] Tìm thấy Email: bewerbung@lidl.de
[27] Tìm thấy Email: bewerbung@lidl.de
[28] Tìm thấy Email: bewerbung@lidl.de
[29] Tìm thấy Email: bewerbung@lidl.de
[30] Tìm thấy Email: bewerbung@lidl.de
[31] Tìm thấy Email: bewerbung@lidl.de
[32] Tìm thấy Email: bewerbung@lidl.de
[33] Tìm thấy Email: bewerbung@lidl.de
[34] Tìm thấy Email: bewerbung@lidl.de
[35] Tìm thấy Email: bewerbung@lidl.de
[36] Tìm thấy Email: bewerbung@lidl.de
[37] Tìm thấy Email: bewerbung@lidl.de
[38] Tìm thấy Email: bewerbung@lidl.de
[39] Tìm thấy Email: bewerbung@lidl.de
[40] Tìm thấy Email: bewerbung@lidl.de
[41] Tìm thấy Email: bewerbung@lidl.de
[42] Tìm thấy Email: bewerbung@lidl.de
[43] Tìm thấy Email: bewerbung@lidl.de
[44] Tìm thấy Email: bewerbung@lidl.de
[45] Tìm thấy Email: bewerbung@lidl.de
[46] Tìm thấy Email: bewer